In [78]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('yearly_flow_data_by_countries.csv').iloc[:, 1:]
df.head(1)

,Economy_Label,Flow_Label,Year,SitcRev3Product_Label,Partner_Label,total_values
0,China,Exports,2016,"Travel goods, handbags & similar containers",Afghanistan,2235


In [79]:
def classify_country(x):
    if x == 'China':
        return 'China'
    elif x == 'India':
        return 'India'
    elif x == 'United States of America':
        return 'United States of America'
    else:
        return 'EU'
def classify_product(x):
    product_info = {'314910': 'textile bag',
                   '322220':'paper bag',
                   '326111':'plastic bag'}
    return product_info.get(x)

def convert_price(row):
    if row['Quantity Description'] == 'number':
        return row['price'] * 1000
    elif row['Quantity Description'] == 'thousand units':
        return row['price']
    elif row['Quantity Description'] == 'kilograms':
        return row['price'] * 127/2
    
def convert_qty(row):
    if row['Quantity Description'] == 'number':
        return row['qty'] / 1000
    elif row['Quantity Description'] == 'thousand units':
        return row['qty']
    elif row['Quantity Description'] == 'kilograms':
        return row['qty'] * 2/127 # 2,000 paper bags weigh 280 pounds https://abcnews.go.com/Technology/story?id=97476&page=1
    
def US_inflation_adjusted(prices):
    inflation_rates = {2020:0.0123,
                      2019:0.0181,
                      2018:0.0244,
                      2017:0.0213,
                      2016:0.0126,
                      2015:0.0012,
                      2014:0.0162}
    new_prices = []
    for year, price in prices:
        orig_year = str(year)
        year = int(year)
        while year != 2021:
            inflation_rate = inflation_rates.get(year)
            if not inflation_rate:
                raise ValueError(f'year {year} is too early, inflation rate not available at this time')
            price *= (1+inflation_rate)
            
            year += 1
        new_prices.append(price)
    return new_prices
US_inflation_adjusted([('2017',100)])

[107.82577193842637]

In [65]:

price = pd.read_csv('price.csv').dropna()
price['NAIC Number'] = price['NAIC Number'].astype(str)
price['product'] = price['NAIC Number'].apply(classify_product)

price = price[((price['NAIC Number']=='314910')&(price['Quantity Description']=='number'))|\
              ((price['NAIC Number']=='322220')&(price['Quantity Description']=='kilograms'))|\
             ((price['NAIC Number']=='326111')&(price['Quantity Description']=='thousand units'))].copy()
price = price.drop(columns = ['Data Type','NAIC Number'])
price = pd.melt(price, id_vars = ['Country','product','Quantity Description'], value_name='price', var_name = 'Year')
price['Year'] = price['Year'].apply(lambda x: x.split(' ')[1])
price['price'] = price.apply(convert_price, axis = 1)
price = price.rename(columns = {'price':'price_per_thousands'})
price['US_inflation_adjusted_price'] = US_inflation_adjusted(price[['Year','price_per_thousands']].values)
price

,Country,product,Quantity Description,Year,price_per_thousands,US_inflation_adjusted_price
0,Austria,textile bag,number,2015,5740.000,6274.703840
1,Austria,paper bag,kilograms,2015,207.645,226.987958
2,Austria,plastic bag,thousand units,2015,35.140,38.413431
3,Belgium,textile bag,number,2015,7940.000,8679.642595
4,Belgium,paper bag,kilograms,2015,455.295,497.707541
...,...,...,...,...,...,...
409,Spain,paper bag,kilograms,2020,149.860,151.703278
410,Spain,plastic bag,thousand units,2020,9.020,9.130946
411,Sweden,textile bag,number,2020,21610.000,21875.803000
412,Sweden,paper bag,kilograms,2020,234.950,237.839885


In [66]:
qty = pd.read_csv('qty.csv')
qty = qty.dropna()
qty['NAIC Number'] = qty['NAIC Number'].astype(int)
qty['NAIC Number'] = qty['NAIC Number'].astype(str)
qty['product'] = qty['NAIC Number'].apply(classify_product)

qty = qty[((qty['NAIC Number']=='314910')&(qty['Quantity Description']=='number'))|\
              ((qty['NAIC Number']=='322220')&(qty['Quantity Description']=='kilograms'))|\
             ((qty['NAIC Number']=='326111')&(qty['Quantity Description']=='thousand units'))].copy()
qty = qty.drop(columns = ['Data Type','NAIC Number'])
qty = pd.melt(qty, id_vars = ['Country','product','Quantity Description'], value_name='qty', var_name = 'Year')
qty['Year'] = qty['Year'].apply(lambda x: x.split(' ')[1])
qty['qty'] = qty.apply(convert_qty, axis = 1)
qty = qty.rename(columns = {'qty':'qty_thousands'})
qty

,Country,product,Quantity Description,Year,qty_thousands
0,Austria,textile bag,number,2015,0.002420
1,Austria,paper bag,kilograms,2015,24.699685
2,Austria,plastic bag,thousand units,2015,3.080000
3,Belgium,textile bag,number,2015,0.000850
4,Belgium,paper bag,kilograms,2015,21.446772
...,...,...,...,...,...
499,Spain,paper bag,kilograms,2020,394.748661
500,Spain,plastic bag,thousand units,2020,594.230000
501,Sweden,textile bag,number,2020,0.001250
502,Sweden,paper bag,kilograms,2020,37.690551


In [67]:
merged = pd.merge(qty,price, on = ['Country', 'product', 'Quantity Description', 'Year'])
merged['country_category'] = merged['Country'].apply(classify_country)
merged = merged.drop(columns = 'Quantity Description')
merged.to_csv('trade_flow_data.csv',index = False)
merged

,Country,product,Year,qty_thousands,price_per_thousands,US_inflation_adjusted_price,country_category
0,Austria,textile bag,2015,0.002420,5740.000,6274.703840,EU
1,Austria,paper bag,2015,24.699685,207.645,226.987958,EU
2,Austria,plastic bag,2015,3.080000,35.140,38.413431,EU
3,Belgium,textile bag,2015,0.000850,7940.000,8679.642595,EU
4,Belgium,paper bag,2015,21.446772,455.295,497.707541,EU
...,...,...,...,...,...,...,...
409,Spain,paper bag,2020,394.748661,149.860,151.703278,EU
410,Spain,plastic bag,2020,594.230000,9.020,9.130946,EU
411,Sweden,textile bag,2020,0.001250,21610.000,21875.803000,EU
412,Sweden,paper bag,2020,37.690551,234.950,237.839885,EU


# total

In [87]:
# qty = pd.read_csv('total_qty.csv')
# qty = qty.dropna()
# qty['NAIC Number'] = qty['NAIC Number'].astype(int)
# qty['NAIC Number'] = qty['NAIC Number'].astype(str)
# qty['product'] = qty['NAIC Number'].apply(classify_product)
# qty = qty[((qty['NAIC Number']=='314910')&(qty['Quantity Description']=='number'))|\
#               ((qty['NAIC Number']=='322220')&(qty['Quantity Description']=='kilograms'))|\
#              ((qty['NAIC Number']=='326111')&(qty['Quantity Description']=='thousand units'))].copy()
# qty = qty.drop(columns = ['Data Type','NAIC Number'])
# qty = pd.melt(qty, id_vars = ['product','Quantity Description'], value_name='qty', var_name = 'Year')
# qty['Year'] = qty['Year'].apply(lambda x: x.split(' ')[1])
# qty['qty'] = qty.apply(convert_qty, axis = 1)
# total_qty = qty.rename(columns = {'qty':'qty_thousands'})
# total_qty

,product,Quantity Description,Year,qty_thousands
0,textile bag,number,2015,2.926166e+05
1,paper bag,kilograms,2015,1.246976e+07
2,plastic bag,thousand units,2015,1.638418e+08
3,textile bag,number,2016,2.757840e+05
4,paper bag,kilograms,2016,1.261258e+07
5,plastic bag,thousand units,2016,1.646017e+08
6,textile bag,number,2017,2.670236e+05
7,paper bag,kilograms,2017,1.267886e+07
8,plastic bag,thousand units,2017,2.280293e+08
9,textile bag,number,2018,2.845897e+05


In [121]:
values = pd.read_csv('total_values.csv')
values = values.groupby(['Data Type','NAIC Number']).sum().reset_index()
values['NAIC Number'] = values['NAIC Number'].astype(str)
values['product'] = values['NAIC Number'].apply(classify_product)
values = values.drop(columns = ['Data Type','NAIC Number'])
total_values = pd.melt(values, id_vars = ['product'], value_name='values', var_name = 'Year')
total_values['Year'] = total_values['Year'].apply(lambda x: x.split(' ')[1])
total_values

,product,Year,values
0,textile bag,2015,1900369946
1,paper bag,2015,3501016887
2,plastic bag,2015,2576392647
3,textile bag,2016,1802486300
4,paper bag,2016,3501091348
5,plastic bag,2016,2615625866
6,textile bag,2017,1900254844
7,paper bag,2017,3599166834
8,plastic bag,2017,2729661777
9,textile bag,2018,2050953721


In [122]:
price = pd.read_csv('total_price.csv')
price['NAIC Number'] = price['NAIC Number'].astype(str)
price['product'] = price['NAIC Number'].apply(classify_product)

price = price[((price['NAIC Number']=='314910')&(price['Quantity Description']=='number'))|\
              ((price['NAIC Number']=='322220')&(price['Quantity Description']=='kilograms'))|\
             ((price['NAIC Number']=='326111')&(price['Quantity Description']=='thousand units'))].copy()
price = price.drop(columns = ['Data Type','NAIC Number'])
price = pd.melt(price, id_vars = ['product','Quantity Description'], value_name='price', var_name = 'Year')
# price
price['Year'] = price['Year'].apply(lambda x: x.split(' ')[1])
price['price'] = price.apply(convert_price, axis = 1)
total_price = price.rename(columns = {'price':'price_per_thousands'})
total_price['US_inflation_adjusted_price'] = US_inflation_adjusted(total_price[['Year','price_per_thousands']].values)
total_price = total_price.drop(columns = ['Quantity Description'])
total_price

,product,Year,price_per_thousands,US_inflation_adjusted_price
0,textile bag,2015,2840.000,3104.557301
1,paper bag,2015,193.675,211.716597
2,plastic bag,2015,13.270,14.506153
3,textile bag,2016,2910.000,3177.265361
4,paper bag,2016,191.770,209.382879
5,plastic bag,2016,13.340,14.565196
6,textile bag,2017,3010.000,3245.555735
7,paper bag,2017,194.310,209.516257
8,plastic bag,2017,9.940,10.717882
9,textile bag,2018,2990.000,3156.751768


In [123]:
merged = pd.merge(total_values,total_price, on = [ 'product', 'Year'])
merged['qty_in_thousands'] = merged['values']/ merged['price_per_thousands']
merged.to_csv('trade_flow_data_total.csv',index = False)
merged

,product,Year,values,price_per_thousands,US_inflation_adjusted_price,qty_in_thousands
0,textile bag,2015,1900369946,2840.000,3104.557301,6.691443e+05
1,paper bag,2015,3501016887,193.675,211.716597,1.807676e+07
2,plastic bag,2015,2576392647,13.270,14.506153,1.941517e+08
3,textile bag,2016,1802486300,2910.000,3177.265361,6.194111e+05
4,paper bag,2016,3501091348,191.770,209.382879,1.825672e+07
5,plastic bag,2016,2615625866,13.340,14.565196,1.960739e+08
6,textile bag,2017,1900254844,3010.000,3245.555735,6.313139e+05
7,paper bag,2017,3599166834,194.310,209.516257,1.852281e+07
8,plastic bag,2017,2729661777,9.940,10.717882,2.746139e+08
9,textile bag,2018,2050953721,2990.000,3156.751768,6.859377e+05


# all country separated

In [125]:
# qty = pd.read_csv('all_country_qty.csv')
# qty = qty.dropna()
# qty['NAIC Number'] = qty['NAIC Number'].astype(int)
# qty['NAIC Number'] = qty['NAIC Number'].astype(str)
# qty['product'] = qty['NAIC Number'].apply(classify_product)

# qty = qty[((qty['NAIC Number']=='314910')&(qty['Quantity Description']=='number'))|\
#               ((qty['NAIC Number']=='322220')&(qty['Quantity Description']=='kilograms'))|\
#              ((qty['NAIC Number']=='326111')&(qty['Quantity Description']=='thousand units'))].copy()
# qty = qty.drop(columns = ['Data Type','NAIC Number'])
# qty = pd.melt(qty, id_vars = ['Country','product','Quantity Description'], value_name='qty', var_name = 'Year')
# qty['Year'] = qty['Year'].apply(lambda x: x.split(' ')[1])
# qty['qty'] = qty.apply(convert_qty, axis = 1)
# qty = qty.rename(columns = {'qty':'qty_thousands'})
# qty

In [126]:
values = pd.read_csv('all_country_values.csv')
values = values.groupby(['Country','NAIC Number']).sum().reset_index()
values['NAIC Number'] = values['NAIC Number'].astype(int)
values['NAIC Number'] = values['NAIC Number'].astype(str)
values['product'] = values['NAIC Number'].apply(classify_product)
values = values.drop(columns = ['NAIC Number'])
values = pd.melt(values, id_vars = ['Country','product'], value_name='values', var_name = 'Year')
values['Year'] = values['Year'].apply(lambda x: x.split(' ')[1])
values

,Country,product,Year,values
0,Afghanistan,textile bag,2015,0
1,Afghanistan,paper bag,2015,0
2,Albania,textile bag,2015,0
3,Albania,paper bag,2015,42336
4,Albania,plastic bag,2015,0
...,...,...,...,...
2623,Vietnam,paper bag,2020,105264040
2624,Vietnam,plastic bag,2020,178793384
2625,Yemen,textile bag,2020,547
2626,Zambia,textile bag,2020,0


In [127]:
price = pd.read_csv('all_country_price.csv').dropna()
price['NAIC Number'] = price['NAIC Number'].astype(str)
price['product'] = price['NAIC Number'].apply(classify_product)

price = price[((price['NAIC Number']=='314910')&(price['Quantity Description']=='number'))|\
              ((price['NAIC Number']=='322220')&(price['Quantity Description']=='kilograms'))|\
             ((price['NAIC Number']=='326111')&(price['Quantity Description']=='thousand units'))].copy()
price = price.drop(columns = ['Data Type','NAIC Number'])
price = pd.melt(price, id_vars = ['Country','product','Quantity Description'], value_name='price', var_name = 'Year')
price['Year'] = price['Year'].apply(lambda x: x.split(' ')[1])
price['price'] = price.apply(convert_price, axis = 1)
price = price.rename(columns = {'price':'price_per_thousands'})
price['US_inflation_adjusted_price'] = US_inflation_adjusted(price[['Year','price_per_thousands']].values)
price = price[price.price_per_thousands != 0].copy()
price = price.drop(columns = ['Quantity Description'])
price.head()

,Country,product,Year,price_per_thousands,US_inflation_adjusted_price
0,Argentina,paper bag,2015,147.320,161.043444
1,Argentina,plastic bag,2015,955.760,1044.792847
2,Australia,textile bag,2015,18630.000,20365.458632
3,Australia,paper bag,2015,960.755,1050.253151
4,Australia,plastic bag,2015,59.460,64.998936


In [128]:
merged = pd.merge(values,price, on = ['Country', 'product',  'Year'])
# merged['country_category'] = merged['Country'].apply(classify_country)
merged['qty_in_thousands'] = merged['values']/ merged['price_per_thousands']
merged.to_csv('trade_flow_data_all_country.csv',index = False)
merged

,Country,product,Year,values,price_per_thousands,US_inflation_adjusted_price,qty_in_thousands
0,Argentina,paper bag,2015,301124,147.320,161.043444,2.044013e+03
1,Argentina,plastic bag,2015,532558,955.760,1044.792847,5.572089e+02
2,Australia,textile bag,2015,584209,18630.000,20365.458632,3.135851e+01
3,Australia,paper bag,2015,618754,960.755,1050.253151,6.440289e+02
4,Australia,plastic bag,2015,2416467,59.460,64.998936,4.064021e+04
...,...,...,...,...,...,...,...
1141,United Kingdom,plastic bag,2020,15240233,32.450,32.849135,4.696528e+05
1142,Venezuela,plastic bag,2020,152398,0.740,0.749102,2.059432e+05
1143,Vietnam,textile bag,2020,147321484,3680.000,3725.264000,4.003301e+04
1144,Vietnam,paper bag,2020,105264040,206.375,208.913412,5.100620e+05
